## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Weather
0,0,Tiksi,RU,71.6872,128.8694,41.50,74,100,5.77,overcast clouds
1,1,Georgetown,MY,5.4112,100.3354,84.13,70,20,4.61,few clouds
2,2,Saskylakh,RU,71.9167,114.0833,63.39,52,100,9.01,overcast clouds
3,3,Vila Velha,BR,-20.3297,-40.2925,84.15,61,0,10.36,clear sky
4,4,Krasnogorsk,RU,55.8225,37.3181,87.80,51,40,11.18,light intensity shower rain


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# Filter the dataset to find the cities that fit the criteria.
clean_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)].copy()


In [5]:
# Check if there are any null values
clean_df.count()

Unnamed: 0         347
City               347
Country            341
Lat                347
Lng                347
Max Temp           347
Humidity           347
Cloudiness         347
Wind Speed         347
Current Weather    347
dtype: int64

In [6]:
# Drop null rows
clean_df.dropna(inplace=True)


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Weather", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Weather,Lat,Lng,Hotel Name
1,Georgetown,MY,84.13,few clouds,5.4112,100.3354,
3,Vila Velha,BR,84.15,clear sky,-20.3297,-40.2925,
4,Krasnogorsk,RU,87.80,light intensity shower rain,55.8225,37.3181,
5,Nikolskoye,RU,82.29,clear sky,59.7035,30.7861,
9,Vimperk,CZ,76.28,few clouds,49.0586,13.7829,
11,Lagos,NG,84.49,overcast clouds,6.5833,3.7500,
20,Arraial Do Cabo,BR,75.06,few clouds,-22.9661,-42.0278,
23,Sri Aman,MY,77.07,scattered clouds,1.2376,111.4621,
24,Kruisfontein,ZA,76.89,clear sky,-34.0033,24.7314,
27,Darab,IR,77.38,overcast clouds,28.7519,54.5444,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.replace(r'^\s*$', np.nan, regex=True)
clean_hotel_df.dropna(inplace=True)
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Weather,Lat,Lng,Hotel Name
1,Georgetown,MY,84.13,few clouds,5.4112,100.3354,Cititel Penang
3,Vila Velha,BR,84.15,clear sky,-20.3297,-40.2925,Quality Suites Vila Velha
4,Krasnogorsk,RU,87.80,light intensity shower rain,55.8225,37.3181,"Hotel ""Academy"""
5,Nikolskoye,RU,82.29,clear sky,59.7035,30.7861,Tourist House - Sablino
9,Vimperk,CZ,76.28,few clouds,49.0586,13.7829,Hotel Zlatá Hvězda
20,Arraial Do Cabo,BR,75.06,few clouds,-22.9661,-42.0278,Pousada Porto Praia
23,Sri Aman,MY,77.07,scattered clouds,1.2376,111.4621,Seri Simanggang Hotel
24,Kruisfontein,ZA,76.89,clear sky,-34.0033,24.7314,Oyster Bay House Rental
27,Darab,IR,77.38,overcast clouds,28.7519,54.5444,Naghshe shapoor Hotel
29,Ibadan,NG,84.97,overcast clouds,7.3878,3.8964,Majerita Hotel And Suites


In [10]:
# 8a. Create the output File (CSV)
output_data_file = ("WeatherPy_vacation.csv")
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [13]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Max Temp} °F + {Current Weather}</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [15]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations,info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))